In [1]:
import random
import copy
from collections import deque
import itertools

import gym
from gym.spaces.box import Box
from gym import wrappers
from gym.wrappers import TransformObservation

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, IterableDataset
import numpy as np

from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

from gym.wrappers import RecordVideo, RecordEpisodeStatistics, TimeLimit, AtariPreprocessing


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

D:\multi_agents_rl\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class DRQN(nn.Module):
    def __init__(self, state_size , n_actions):
        super(DRQN, self).__init__()
        
        self.state_size = state_size
        self.conv = nn.Sequential(
                        nn.Conv2d(state_size[0], 32, 3, stride=2, padding=1),
                        nn.ELU(),
                        nn.Conv2d(32, 32, 3, stride=2, padding=1),
                        nn.ELU(),
                        nn.Conv2d(32, 32, 3, stride=2, padding=1),
                        nn.ELU(),
                        nn.Conv2d(32, 32, 3, stride=2, padding=1),
                        nn.ELU()
                    )
        conv_out_size = self._get_conv_out(state_size)
        self.lstm = nn.LSTMCell(conv_out_size, 256)
        self.fc_adv = nn.Linear(256, n_actions) 
        self.fc_value = nn.Linear(256, 1)
        
    def _get_conv_out(self, shape):
        conv_out = self.conv(torch.zeros(1, *shape))
        return int(np.prod(conv_out.size()))
    
    def forward(self, x, prev_state=None):        
        o = self.conv(x.float()).view(x.shape[0], -1)
        if prev_state is not None:
            hs, ct = self.lstm(o, prev_state)
        else:
            hs = torch.zeros(x.shape[0], 256).to(device)
            ct = torch.zeros(x.shape[0], 256).to(device)
            hs, ct = self.lstm(o)    
        
        adv = self.fc_adv(hs)
        value = self.fc_value(hs)  
        return value + adv - torch.mean(adv, dim=1, keepdim=True), (hs, ct)

In [3]:
def epsilon_greedy(state, env, net, hidden=None, epsilon=0.0):
    if np.random.random() < epsilon:
        action = env.action_space.sample()
    else:
        state = state.to(device)
        q_values, hidden = net(state, hidden)
        _, action = torch.max(q_values, dim=1)
        action = int(action.item())
    return action, hidden

In [4]:
def slice_deque(buffer, start, stop, step):
    buffer.rotate(-start)
    slice = list(itertools.islice(buffer, 0, stop-start, step))
    buffer.rotate(start)
    return slice

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
        
    def __len__(self):
        return len(self.buffer)
    
    def append(self, experience):
        self.buffer.append(experience)
        
    def sample(self, sample_size, sequence_len):
        batch = []
        while len(batch) < int(sample_size / 2) :
            start = random.randint(0, len(self.buffer) - sequence_len)
            sequenze_sample = slice_deque(self.buffer, start, start + sequence_len, 1)
            for sample in sequenze_sample:
                if sample[1] == 1 or sample[1] == -1:
                    batch += sequenze_sample
                    
        while len(batch) < sample_size:
            start = random.randint(0, len(self.buffer) - sequence_len)
            sequenze_sample = slice_deque(self.buffer, start, start + sequence_len, 1)
            batch += sequenze_sample
                    
        return batch

In [5]:
class RLDataset(IterableDataset):
    def __init__(self, buffer, sample_size=400, sequence_len=5):
        self.buffer = buffer
        self.sample_size = sample_size
        self.sequence_len = sequence_len
        
    def __iter__(self):
        for experience in self.buffer.sample(self.sample_size,self.sequence_len):
            yield experience

In [6]:
def create_environment(name):
    env = gym.make(name, render_mode="rgb_array")
    env.unwrapped._frameskip = 1
    env = TimeLimit(env, max_episode_steps=1000)
    env = RecordVideo(env, video_folder='./videos/drqn-pong', episode_trigger=lambda x: x % 50 == 0)
    env = RecordEpisodeStatistics(env)
    env = gym.wrappers.AtariPreprocessing(env, frame_skip=8, noop_max=28, screen_size=64, terminal_on_life_loss=False, grayscale_obs=True, grayscale_newaxis=False, scale_obs=True)
    env.observation_space = Box(0.0, 1.0, [1, 64, 64])
    return env



In [7]:
class DeepQLearning(LightningModule):
    def __init__(self, env_name, policy=epsilon_greedy, capacity=100_000, 
               batch_size=256, lr=1e-3, hidden_size=128, gamma=0.99, 
               loss_fn=nn.MSELoss(), optim=AdamW, eps_start=1.0, eps_end=0.15, 
               eps_last_episode=400, samples_per_epoch=1024, sync_rate=10,
               sequence_length = 8):
    
        super().__init__()
        self.env = create_environment(env_name)

        obs_size = self.env.observation_space.shape
        n_actions = self.env.action_space.n

        self.q_net = DRQN(obs_size, n_actions)

        self.target_q_net = copy.deepcopy(self.q_net)

        self.policy = policy
        self.buffer = ReplayBuffer(capacity=capacity)
        self.save_hyperparameters()

        while len(self.buffer) < self.hparams.samples_per_epoch:
            print(f"{len(self.buffer)} samples in experience buffer. Filling...")
            self.play_episode(epsilon=self.hparams.eps_start)
            
    @torch.no_grad()
    def play_episode(self, policy=None, epsilon=0.):
        state  = self.env.reset()
        state  = torch.from_numpy(state[0]).unsqueeze(dim=0)
        hidden = None
        done = False
        
        while not done:
            if policy:
                action, hidden = policy(state.unsqueeze(dim=0), self.env, self.q_net, hidden, epsilon=epsilon)
            else:
                action = self.env.action_space.sample()
            next_state, reward, done, tru , _ = self.env.step(action)
            if tru:
                done = tru
            
            next_state = torch.from_numpy(next_state).unsqueeze(dim=0) 
            exp = (state, action, reward, done, next_state)
            
            self.buffer.append(exp)
            state = next_state
            
        self.env.close()
        
        
    def forward(self, x):
        return self.q_net(x)

    
    def configure_optimizers(self):
        q_net_optimizer = self.hparams.optim(self.q_net.parameters(), lr=self.hparams.lr)
        return [q_net_optimizer]

     # Create dataloader.
    def train_dataloader(self):
        dataset = RLDataset(self.buffer, self.hparams.samples_per_epoch, self.hparams.sequence_length)
       
        dataloader = DataLoader(
            dataset=dataset,
            batch_size=self.hparams.batch_size * self.hparams.sequence_length
        )
        return dataloader
    
    def training_step(self, batch, batch_idx):
        states, actions, rewards, dones, next_states = batch
        actions = actions.unsqueeze(1)
        rewards = rewards.unsqueeze(1)
        dones = dones.unsqueeze(1)
        
        stack_q_values = []
        stack_actions = []
        stack_rewards = []
        stack_dones = []
        hidden = None
        for i in range(self.hparams.sequence_length):
            stack_actions.append(actions[i::self.hparams.sequence_length])
            stack_dones.append(dones[i::self.hparams.sequence_length])
            stack_rewards.append(rewards[i::self.hparams.sequence_length])
            q_values, hidden  = self.q_net(states[i::self.hparams.sequence_length],hidden)
            stack_q_values.append(q_values)
        stack_q_values = torch.cat(stack_q_values,dim=0)
        stack_actions = torch.cat(stack_actions,dim=0)
        stack_dones = torch.cat(stack_dones,dim=0)
        stack_rewards = torch.cat(stack_rewards,dim=0)

    
        state_action_values = torch.gather(stack_q_values, -1, stack_actions)
        
        
        with torch.no_grad():
            stack_next_q_values = []
            hidden = None
            for i in range(self.hparams.sequence_length):
                q_values, hidden = self.target_q_net(next_states[i::self.hparams.sequence_length],hidden)
                stack_next_q_values.append(q_values)
                
        stack_next_q_values = torch.cat(stack_next_q_values,dim=0)
        next_action_values = torch.max(stack_next_q_values, dim=1)[0].unsqueeze(dim=1)
        next_action_values[stack_dones] = 0.0

        expected_state_action_values = stack_rewards + self.hparams.gamma * next_action_values
        
       
        loss = self.hparams.loss_fn(state_action_values.float(), expected_state_action_values.float())
        self.log('episode/Q-Error', loss)
        return loss
    
    # Training epoch end.
    def training_epoch_end(self, training_step_outputs):
        epsilon = max(
            self.hparams.eps_end,
            self.hparams.eps_start - self.current_epoch / self.hparams.eps_last_episode
        )

        self.play_episode(policy=self.policy, epsilon=epsilon)
        self.log('episode/Return', self.env.return_queue[-1])

        if self.current_epoch % self.hparams.sync_rate == 0:
            self.target_q_net.load_state_dict(self.q_net.state_dict())
            
            
    def save_model(self):
        torch.save(self.q_net.state_dict(), "./model")
        
    def load_model(self):
        self.q_net.load_state_dict(torch.load( "./model"))


In [8]:
algo = DeepQLearning('ALE/Pong-v5')

checkpoint_callback = ModelCheckpoint(dirpath="./checkpoints/drqb-pong", save_top_k=1,mode="max", monitor="episode/Return")

trainer = Trainer(
     accelerator='gpu',
     devices=num_gpus,
     max_epochs=10_000,
     callbacks=[checkpoint_callback], # EarlyStopping(monitor='episode/Return', mode='max', patience=1000)
)

trainer.fit(algo)

D:\multi_agents_rl\env\lib\site-packages\gym\wrappers\record_video.py:75: UserWarning: WARN: Overwriting existing videos at D:\multi_agents_rl\videos\drqn-pong folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
D:\multi_agents_rl\env\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(
D:\multi_agents_rl\env\lib\site-packages\gym\wrappers\monitoring\video_recorder.py:59: UserWarning: WARN: Disabling video recorder because environment <TimeLimit<OrderEnforcing<PassiveEnvChecker<AtariEnv<ALE/Pong-v5>>>>> was not initialized with any compatible video mode between `rgb_array` and `rgb_array_list`
  logger.warn(
D:\multi_agents_rl\env\lib\site-packages\gym\utils\passive_env_checker.py:289: UserWarning: WARN

0 samples in experience buffer. Filling...
123 samples in experience buffer. Filling...
245 samples in experience buffer. Filling...
369 samples in experience buffer. Filling...
494 samples in experience buffer. Filling...
617 samples in experience buffer. Filling...
740 samples in experience buffer. Filling...
865 samples in experience buffer. Filling...
987 samples in experience buffer. Filling...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
D:\multi_agents_rl\env\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory D:\multi_agents_rl\checkpoints\drqb-pong exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type | Params
--------------------------------------
0 | q_net        | DRQN | 818 K 
1 | target_q_net | DRQN | 818 K 
--------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.547     Total estimated model params size (MB)
D:\multi_agents_rl\env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasi

Epoch 9999: : 1it [00:00,  1.42it/s, loss=0.0114, v_num=7]

`Trainer.fit` stopped: `max_epochs=10000` reached.


Epoch 9999: : 1it [00:00,  1.38it/s, loss=0.0114, v_num=7]


In [9]:
# PLay a game with the algorithm
env = algo.env
policy = algo.policy
q_net = algo.q_net.cuda()
frames = []

for episode in range(10):
    state = env.reset()
    state = state[0]
    state = torch.from_numpy(state).unsqueeze(dim=0)
    done = False
    hidden = None
    while not done:
        action, hidden = policy(state, env, q_net, epsilon=0, hidden=hidden)
        next_state, reward, done, tru , _ = env.step(action)
        next_state = torch.from_numpy(next_state).unsqueeze(dim=0)
        if tru:
            done = tru
        state = next_state
        frame = state.squeeze(dim=0)
        frame = frame.numpy()
        frames.append(frame)
        


In [ ]:
# Play a random game
frames = []
env = create_environment("ALE/Pong-v5")
for episode in range(1):
  done = False
  obs, _  = env.reset()
  while not done:
    frames.append(obs)
    action = env.action_space.sample()
    obs, reward, done, t ,_ = env.step(action)
    if t:
        done = True
    

In [11]:
import matplotlib.pyplot as plt
from IPython.display import clear_output


for frame in frames:
    plt.imshow(frame)
    plt.show()
    clear_output(wait=True)


KeyboardInterrupt: 